In [1]:
import pandas as pd
import re

In [2]:
df_raw = pd.read_csv("C:/Users/Janak ARORA/Desktop/Ironhack_files/Week_7/Day1/machine_learning_project/data/raw/Amazon.csv")
df_raw.head()

C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\2142167822.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("C:/Users/Janak ARORA/Desktop/Ironhack_files/Week_7/Day1/machine_learning_project/data/raw/Amazon.csv")


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [3]:
df_raw.dtypes

index                   int64
Order ID               object
Date                   object
Status                 object
Fulfilment             object
Sales Channel          object
ship-service-level     object
Style                  object
SKU                    object
Category               object
Size                   object
ASIN                   object
Courier Status         object
Qty                     int64
currency               object
Amount                float64
ship-city              object
ship-state             object
ship-postal-code      float64
ship-country           object
promotion-ids          object
B2B                      bool
fulfilled-by           object
Unnamed: 22            object
dtype: object

In [4]:
df_raw.shape

(128975, 24)

In [5]:
df_raw.columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object')

In [6]:
df_raw['Qty'].unique()

array([ 0,  1,  2, 15,  3,  9, 13,  5,  4,  8])

In [7]:
df_raw = df_raw.drop(columns=["index", "Unnamed: 22", "Sales Channel ", "currency", "fulfilled-by", "ship-country", "promotion-ids"])
df_raw.head()

,Order ID,Date,Status,Fulfilment,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,B2B
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,647.62,MUMBAI,MAHARASHTRA,400081.0,False
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,406.00,BENGALURU,KARNATAKA,560085.0,False
2,404-0687676-7273146,04-30-22,Shipped,Amazon,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,True
3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,False
4,407-1069790-7240320,04-30-22,Shipped,Amazon,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,574.00,CHENNAI,TAMIL NADU,600073.0,False


In [8]:
df_raw.isnull().sum()

Order ID                 0
Date                     0
Status                   0
Fulfilment               0
ship-service-level       0
Style                    0
SKU                      0
Category                 0
Size                     0
ASIN                     0
Courier Status        6872
Qty                      0
Amount                7795
ship-city               33
ship-state              33
ship-postal-code        33
B2B                      0
dtype: int64

In [9]:
df_cleaned = df_raw.dropna()

In [10]:
df_cleaned.isnull().sum()

Order ID              0
Date                  0
Status                0
Fulfilment            0
ship-service-level    0
Style                 0
SKU                   0
Category              0
Size                  0
ASIN                  0
Courier Status        0
Qty                   0
Amount                0
ship-city             0
ship-state            0
ship-postal-code      0
B2B                   0
dtype: int64

In [11]:
df_cleaned.shape

(116016, 17)

In [12]:
df_cleaned["Date"] = pd.to_datetime(df_cleaned["Date"], errors="coerce")

C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\3508508436.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned["Date"] = pd.to_datetime(df_cleaned["Date"], errors="coerce")
C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\3508508436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["Date"] = pd.to_datetime(df_cleaned["Date"], errors="coerce")


In [13]:
df_cleaned.dtypes

Order ID                      object
Date                  datetime64[ns]
Status                        object
Fulfilment                    object
ship-service-level            object
Style                         object
SKU                           object
Category                      object
Size                          object
ASIN                          object
Courier Status                object
Qty                            int64
Amount                       float64
ship-city                     object
ship-state                    object
ship-postal-code             float64
B2B                             bool
dtype: object

In [14]:
df_cleaned["Order ID"] = df_cleaned["Order ID"].str.replace("-", "", regex=False)

C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\2792898832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["Order ID"] = df_cleaned["Order ID"].str.replace("-", "", regex=False)


In [15]:
df_cleaned.head()

,Order ID,Date,Status,Fulfilment,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,B2B
1,17191981511101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,406.0,BENGALURU,KARNATAKA,560085.0,False
2,40406876767273146,2022-04-30,Shipped,Amazon,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,329.0,NAVI MUMBAI,MAHARASHTRA,410210.0,True
4,40710697907240320,2022-04-30,Shipped,Amazon,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,574.0,CHENNAI,TAMIL NADU,600073.0,False
5,40414909844578765,2022-04-30,Shipped,Amazon,Expedited,SET264,SET264-KR-NP-XL,Set,XL,B08YN7XDSG,Shipped,1,824.0,GHAZIABAD,UTTAR PRADESH,201102.0,False
6,40857484996859555,2022-04-30,Shipped,Amazon,Expedited,J0095,J0095-SET-L,Set,L,B08CMHNWBN,Shipped,1,653.0,CHANDIGARH,CHANDIGARH,160036.0,False


In [16]:
df_cleaned.to_csv("cleaned_amazon_data.csv", index=False)

### Clean City Column

In [17]:
df_cleaned["ship-city"].unique()

array(['BENGALURU', 'NAVI MUMBAI', 'CHENNAI', ...,
       'GULABPURA, Distt BHILWARA', 'Prayagraj (ALLAHABAD)', 'Halol'],
      shape=(8499,), dtype=object)

In [18]:
unique_cities = sorted(df_cleaned["ship-city"].dropna().unique())
len(unique_cities), unique_cities[:50]

(8499,
 ['(Chikmagalur disterict).     (N.R pur thaluku)',
  '(Via Cuncolim)Quepem,South Goa',
  ',HYDERABAD',
  ',raibarely road faizabad (Ayodhya)',
  '..katra',
  '.Gannavaram',
  '.azamgarh',
  '1',
  '116  B. P. M. B SARANI, UTTARPARA KOTRUNG',
  '6th mile tadong',
  '7BARASAT',
  'AAMBY VALLEY',
  'ABHAYAPURI',
  'ABHAYAPURI, Bongaigaon',
  'ABOHAR',
  'ABU ROAD',
  'ACHALPUR',
  'ACHAMPET',
  'ACHANTA',
  'ACHARAPAKKAM',
  'ACHHNERA',
  'ADALAJ',
  'ADAMPUR',
  'ADAPUR',
  'ADBHAR',
  'ADDAKAL',
  'ADDANKI',
  'ADICHANALLOOR',
  'ADILABAD',
  'ADINAD',
  'ADIPUR, GANDHIDHAM',
  'ADITYANA',
  'ADITYAPUR',
  'ADONI',
  'ADOOR',
  'ADOOR, vaddakkedathu cavu',
  'ADRA',
  'AFZALGARH',
  'AGAR',
  'AGAR Malwa',
  'AGARTALA',
  'AGATTI',
  'AGIRIPALLE',
  'AGRA',
  'AGRA 282005',
  'AHERI',
  'AHIWARA',
  'AHMADNAGAR',
  'AHMADNAGARA',
  'AHMADPUR'])

In [19]:
# Step 1: normalize city text
def normalize_city(city):
    city = str(city).lower().strip()
    city = re.sub(r"[^a-z\s]", " ", city)   # remove symbols/numbers
    city = re.sub(r"\s+", " ", city)        # collapse spaces
    return city

def camel_with_space(city):
    return " ".join(word.capitalize() for word in city.split())
    
# Step 2: explicit duplicate / variant city merges
city_mapping = {
    # Bangalore
    "bengaluru": "bangalore",
    "banglore": "bangalore",
    "blr": "bangalore",

    # Delhi NCR
    "new delhi": "new delhi",
    "delhi ncr": "delhi",
    "ncr": "delhi",

    # Mumbai
    "bombay": "mumbai",
    "andar": "mumbai",
    "andhari": "mumbai",
    "andheri": "mumbai",
    "andheri e": "mumbai",
    "andheri east": "mumbai",
    "andheri east mumbai": "mumbai",
    "andheri west mumbai": "mumbai",
    
    # Chennai
    "madras": "chennai",

    # Kolkata
    "calcutta": "kolkata",

    # Hyderabad
    "secunderabad": "hyderabad",

    # Kerala
    "cochin": "kochi",
    "trivandrum": "thiruvananthapuram",

    # Gurgaon / Gurugram
    "gurugram": "gurgaon",

    # Noida
    "greater noida": "noida",
    "noida greater noida": "noida"
}

df_cleaned["ship-city"] = df_cleaned["ship-city"].apply(normalize_city)
df_cleaned["ship-city"] = df_cleaned["ship-city"].replace(city_mapping)
df_cleaned["ship-city"] = df_cleaned["ship-city"].apply(camel_with_space)

C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\2987929465.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["ship-city"] = df_cleaned["ship-city"].apply(normalize_city)
C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\2987929465.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["ship-city"] = df_cleaned["ship-city"].replace(city_mapping)
C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\2987929465.py:56: SettingWithCopyWarning: 
A value is trying to be set o

In [20]:
cleaned_unique_cities = sorted(df_cleaned["ship-city"].unique())

print("Unique cities BEFORE:", len(unique_cities))
print("Unique cities AFTER:", len(cleaned_unique_cities))
cleaned_unique_cities

Unique cities BEFORE: 8499
Unique cities AFTER: 6554


['',
 'Aalo',
 'Aamby Valley',
 'Abhayapuri',
 'Abhayapuri Bongaigaon',
 'Abohar',
 'Abu Road',
 'Aburoad',
 'Achalpur',
 'Achampet',
 'Achanta',
 'Acharapakkam',
 'Achhnera',
 'Adajan Gam Surat',
 'Adalaj',
 'Adambakkam',
 'Adampur',
 'Adapur',
 'Adarsh Nagar Jaipur',
 'Adbhar',
 'Addakal',
 'Addanki',
 'Addanki Mandal Bapatl District',
 'Addision Nagar Mangadu',
 'Adichanalloor',
 'Adilabad',
 'Adimaly',
 'Adinad',
 'Adipur',
 'Adipur Gandhidham',
 'Adityana',
 'Adityapur',
 'Adityapur Jamshedpur',
 'Adoni',
 'Adoor',
 'Adoor Vaddakkedathu Cavu',
 'Adra',
 'Adyal',
 'Afzalgarh',
 'Agar',
 'Agar Malwa',
 'Agarpara',
 'Agartala',
 'Agarwada',
 'Agatti',
 'Agiripalle',
 'Agorpara',
 'Agra',
 'Ahemadabad',
 'Ahemdabad',
 'Ahemedabad',
 'Aheri',
 'Ahiwara',
 'Ahmadabad',
 'Ahmadnagar',
 'Ahmadnagara',
 'Ahmadpur',
 'Ahmdabad',
 'Ahmed Nagar',
 'Ahmedabad',
 'Ahmedabadvasna',
 'Ahmedanagar',
 'Ahmednagar',
 'Ahmedpur',
 'Ahore',
 'Aimanam',
 'Ainavilli Mandal East Godhavari',
 'Airoli',
 '

In [21]:
df_cleaned["ship-city"].value_counts()

ship-city
Bangalore                                 12297
Hyderabad                                  8819
Mumbai                                     6642
New Delhi                                  5767
Chennai                                    5760
                                          ...  
Talcher Thermal Power Station Township        1
Limla                                         1
Marancheri                                    1
Ambepur                                       1
Hyderabad Shamshabad                          1
Name: count, Length: 6554, dtype: int64

### Clean State Column

In [22]:
df_cleaned["ship-state"].unique()

array(['KARNATAKA', 'MAHARASHTRA', 'TAMIL NADU', 'UTTAR PRADESH',
       'CHANDIGARH', 'TELANGANA', 'ANDHRA PRADESH', 'RAJASTHAN', 'DELHI',
       'HARYANA', 'ASSAM', 'JHARKHAND', 'CHHATTISGARH', 'ODISHA',
       'KERALA', 'MADHYA PRADESH', 'WEST BENGAL', 'NAGALAND', 'Gujarat',
       'UTTARAKHAND', 'BIHAR', 'JAMMU & KASHMIR', 'PUNJAB',
       'HIMACHAL PRADESH', 'ARUNACHAL PRADESH', 'MANIPUR', 'Goa',
       'MEGHALAYA', 'PUDUCHERRY', 'GOA', 'TRIPURA', 'LADAKH',
       'DADRA AND NAGAR', 'SIKKIM', 'Delhi', 'ANDAMAN & NICOBAR ',
       'Punjab', 'Rajshthan', 'Manipur', 'rajasthan', 'Odisha', 'NL',
       'Bihar', 'MIZORAM', 'punjab', 'New Delhi', 'Rajasthan',
       'Punjab/Mohali/Zirakpur', 'Puducherry', 'delhi', 'RJ',
       'Chandigarh', 'orissa', 'LAKSHADWEEP', 'goa', 'PB',
       'Arunachal Pradesh', 'AR', 'Pondicherry', 'Sikkim',
       'Arunachal pradesh', 'Nagaland', 'bihar', 'Mizoram', 'rajsthan',
       'Orissa', 'Rajsthan'], dtype=object)

In [23]:
# Capture original unique states
original_states = sorted(df_cleaned["ship-state"].dropna().unique())

# Normalize state names
df_cleaned["ship-state"] = df_cleaned["ship-state"].astype(str).str.strip().str.lower()

# Known manual merges for common duplicates / variants
state_mapping = {
    # Abbreviations
    "pb": "Punjab",
    "rj": "Rajasthan",
    "ar": "Arunachal Pradesh",
    "up": "Uttar Pradesh",
    "tn": "Tamil Nadu",
    "nl": "Nagaland",
    "mh": "Maharashtra",

    # Misspellings / variants
    "rajsthan": "Rajasthan",
    "rajshthan": "Rajasthan",
    "orissa": "Odisha",
    "pondicherry": "Puducherry",
    "nct of delhi": "Delhi",

    # Combined locations
    "punjab/mohali/zirakpur": "punjab",

    # Symbol variants
    "andaman & nicobar": "Andaman And Nicobar",
    "andaman and nicobar islands": "Andaman And Nicobar",
    "dadra & nagar haveli": "Dadra And Nagar Haveli",
    "dadra and nagar haveli": "Dadra And Nagar Haveli",
    "daman & diu": "Daman And Diu",
    "jammu & kashmir": "Jammu And Kashmir",
    "jammu and kashmir": "Jammu And Kashmir",
    "odisha": "Odisha",
    "delhi": "Delhi",
    "rajasthan": "Rajasthan"
}

df_cleaned["ship-state"] = df_cleaned["ship-state"].replace(state_mapping)

def camel_case(text):
    words = text.replace("&", "and").replace("-", " ").split()
    return " ".join(word.capitalize() for word in words)

df_cleaned["ship-state"] = df_cleaned["ship-state"].apply(camel_case)

C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\2026754172.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["ship-state"] = df_cleaned["ship-state"].astype(str).str.strip().str.lower()
C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\2026754172.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["ship-state"] = df_cleaned["ship-state"].replace(state_mapping)
C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\2026754172.py:47: SettingWithCopyWarning: 
A value is 

In [24]:
df_cleaned["ship-state"].unique()

array(['Karnataka', 'Maharashtra', 'Tamil Nadu', 'Uttar Pradesh',
       'Chandigarh', 'Telangana', 'Andhra Pradesh', 'Rajasthan', 'Delhi',
       'Haryana', 'Assam', 'Jharkhand', 'Chhattisgarh', 'Odisha',
       'Kerala', 'Madhya Pradesh', 'West Bengal', 'Nagaland', 'Gujarat',
       'Uttarakhand', 'Bihar', 'Jammu And Kashmir', 'Punjab',
       'Himachal Pradesh', 'Arunachal Pradesh', 'Manipur', 'Goa',
       'Meghalaya', 'Puducherry', 'Tripura', 'Ladakh', 'Dadra And Nagar',
       'Sikkim', 'Andaman And Nicobar', 'Mizoram', 'New Delhi',
       'Lakshadweep'], dtype=object)

## Calculating Sales Amount

In [25]:
# Ensure Qty and Amount are numeric

df_cleaned["Qty"] = pd.to_numeric(df_cleaned["Qty"], errors="coerce")
df_cleaned["Amount"] = pd.to_numeric(df_cleaned["Amount"], errors="coerce")

# Create new column: Sales Amount
df_cleaned["Sales Amount"] = df_cleaned["Qty"] * df_cleaned["Amount"]
df_cleaned.head()

C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\378499708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["Qty"] = pd.to_numeric(df_cleaned["Qty"], errors="coerce")
C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\378499708.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["Amount"] = pd.to_numeric(df_cleaned["Amount"], errors="coerce")
C:\Users\Janak ARORA\AppData\Local\Temp\ipykernel_14988\378499708.py:7: SettingWithCopyWarning: 
A value is trying to be set on a 

,Order ID,Date,Status,Fulfilment,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,B2B,Sales Amount
1,17191981511101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,406.0,Bangalore,Karnataka,560085.0,False,406.0
2,40406876767273146,2022-04-30,Shipped,Amazon,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,329.0,Navi Mumbai,Maharashtra,410210.0,True,329.0
4,40710697907240320,2022-04-30,Shipped,Amazon,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,574.0,Chennai,Tamil Nadu,600073.0,False,574.0
5,40414909844578765,2022-04-30,Shipped,Amazon,Expedited,SET264,SET264-KR-NP-XL,Set,XL,B08YN7XDSG,Shipped,1,824.0,Ghaziabad,Uttar Pradesh,201102.0,False,824.0
6,40857484996859555,2022-04-30,Shipped,Amazon,Expedited,J0095,J0095-SET-L,Set,L,B08CMHNWBN,Shipped,1,653.0,Chandigarh,Chandigarh,160036.0,False,653.0


In [26]:
numerical_cols = df_cleaned.select_dtypes(include='number')
numerical_cols

,Qty,Amount,ship-postal-code,Sales Amount
1,1,406.0,560085.0,406.0
2,1,329.0,410210.0,329.0
4,1,574.0,600073.0,574.0
5,1,824.0,201102.0,824.0
6,1,653.0,160036.0,653.0
...,...,...,...,...
128970,1,517.0,500013.0,517.0
128971,1,999.0,122004.0,999.0
128972,1,690.0,500049.0,690.0
128973,1,1199.0,389350.0,1199.0


#### Train Test Split

In [27]:
features = df_cleaned.drop(columns = ["Sales Amount"])
target = df_cleaned['Sales Amount']
print(target.head())

1    406.0
2    329.0
4    574.0
5    824.0
6    653.0
Name: Sales Amount, dtype: float64


In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [30]:
X_train.head()

,Order ID,Date,Status,Fulfilment,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,Amount,ship-city,ship-state,ship-postal-code,B2B
55752,40616194968051549,2022-05-26,Shipped,Amazon,Expedited,J0301,J0301-TP-XXL,Top,XXL,B099S8X47K,Shipped,1,518.0,Nashik,Maharashtra,422009.0,False
105768,40889291681804365,2022-06-17,Shipped - Delivered to Buyer,Merchant,Standard,JNE3798,JNE3798-KR-A-M,Western Dress,M,B09TH54761,Shipped,1,771.0,Duliajan,Assam,786602.0,False
116594,40239092481349115,2022-06-09,Shipped - Delivered to Buyer,Merchant,Standard,SET198,SET198-KR-NP-A-L,Set,L,B08L934XZ9,Shipped,1,523.0,Dhanbad,Jharkhand,826005.0,False
125405,40525720817062735,2022-06-03,Shipped,Amazon,Expedited,SET341,SET341-KR-NP-XL,Set,XL,B09NPX3HG3,Shipped,1,857.0,Tinsukia,Assam,786125.0,False
68137,40889515604200348,2022-05-16,Shipped - Delivered to Buyer,Merchant,Standard,J0005,J0005-DR-XXL,Western Dress,XXL,B08B3GTC12,Shipped,1,899.0,Lucknow,Uttar Pradesh,226031.0,False


In [31]:
y_train.head()

55752     518.0
105768    771.0
116594    523.0
125405    857.0
68137     899.0
Name: Sales Amount, dtype: float64

In [32]:
from sklearn.neighbors import KNeighborsRegressor

In [33]:
knn = KNeighborsRegressor(n_neighbors=10) # K = 10

In [35]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92812 entries, 55752 to 76164
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Order ID            92812 non-null  object        
 1   Date                92812 non-null  datetime64[ns]
 2   Status              92812 non-null  object        
 3   Fulfilment          92812 non-null  object        
 4   ship-service-level  92812 non-null  object        
 5   Style               92812 non-null  object        
 6   SKU                 92812 non-null  object        
 7   Category            92812 non-null  object        
 8   Size                92812 non-null  object        
 9   ASIN                92812 non-null  object        
 10  Courier Status      92812 non-null  object        
 11  Qty                 92812 non-null  int64         
 12  Amount              92812 non-null  float64       
 13  ship-city           92812 non-null  object     

In [34]:
knn.fit(X_train, y_train)

TypeError: Cannot cast DatetimeArray to dtype float64

In [ ]:
print(f"The R2 of the model is {knn.score(X_test, y_test): .2f}")

In [ ]:
knn.predict(X_test)

In [ ]:
y_test.values